# A2C - CartPole
Implementation of the A2C RL Algorithm for the OpenAI's Gym environment CartPole-V1 (not in parallel).

In [4]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plot

import tensorflow as tf

import gym

In [18]:
# Set up the environment
env = gym.make('CartPole-v0')
env.reset()
for _ in range(100):
   env.render()
   env.step(env.action_space.sample()) # take a random action
env.close()

In [17]:
from tensorflow.keras.layers import Layer

relu = tf.keras.activations.relu


SyntaxError: positional argument follows keyword argument (<ipython-input-17-1de310187a15>, line 10)

In [19]:
!pip install 'gym[all]'

     |████████████████████████████████| 3.3MB 307kB/s eta 0:00:01
     |████████████████████████████████| 47.9MB 87kB/s  eta 0:00:01
     |████████████████████████████████| 122kB 103kB/s eta 0:00:01
     |████████████████████████████████| 3.5MB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 942kB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 2.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 2.5MB/s ta 0:00:01
     |████████████████████████████████| 1.9MB 1.3MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /opt/miniconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/jf/tqb248c95kl2zz3bft_8ncnr0000gn/T/pip-install-1mwz0lv_/mujoco-py/setup.py'"'"'; __file__='"'"'/private/var/folders/jf/tqb248c95kl2zz3bft_8ncnr0000gn/T/pip-install-1mwz0lv_/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f

ERROR: Command errored out with exit status 1: /opt/miniconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/jf/tqb248c95kl2zz3bft_8ncnr0000gn/T/pip-install-1mwz0lv_/mujoco-py/setup.py'"'"'; __file__='"'"'/private/var/folders/jf/tqb248c95kl2zz3bft_8ncnr0000gn/T/pip-install-1mwz0lv_/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/jf/tqb248c95kl2zz3bft_8ncnr0000gn/T/pip-record-no2x2b2g/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
